# SOFT-CP

1.生成source和softlabel的路径list<br/>
2.分别随机从两个list选取source和softlabel<br/>
3.随机从tansform中选取多个变换对softlabel和source进行变换<br/>
4.将softlabel resize成source大小，进行paste<br/>
5.将合成图进行随机的翻转，模糊，尺度等操作

## PH

In [11]:


#生成source和softlabel的路径list
import os
source_img_path="D:\\dataaug\\paper\\figure3\\AUG\\image"
source_lab_path="D:\\dataaug\\paper\\figure3\\AUG\\label"
softlab_path="D:\\dataaug\\paper\\figure3\\AUG\\soft_lab"
softimg_path="D:\\dataaug\\paper\\figure3\\AUG\\soft_img"

def path_zip(imgs_path,
             labs_path):
    list_zip=[]
    img_names=os.listdir(imgs_path)
    lab_names=os.listdir(labs_path)
    assert(len(img_names)==len(lab_names))
    for i in range(len(img_names)):
        assert(img_names[i].split("_")[0]==lab_names[i].split("_")[0])
        img_path=imgs_path+"\\"+img_names[i]
        lab_path=labs_path+"\\"+lab_names[i]
        list_zip.append([img_path,lab_path])
    return list_zip

In [12]:
#分别随机从两个list选取source和softlabel
import numpy as np
def Random_select(path_zip):
    L=len(path_zip)
    num=np.random.randint(0,L)
    return path_zip[num]

In [13]:
import cv2
import numpy as np

#仿射变换
def affine(img):
    pts1 = np.float32([[1,0],[0,0],[0,1]])
    pts2 = np.float32([[1.2,0.1],[0,0.1],[0.2,1.2]])
    M = cv2.getAffineTransform(pts1,pts2)
    cols, rows=img.shape[:2]
    img_T = cv2.warpAffine(img, M, (rows, cols))
    return img_T

def rotate(img,angle):#angle=[-180:180]
    cols, rows=img.shape[:2]
    M=cv2.getRotationMatrix2D((rows/2,cols/2),angle,1)
    img_T = cv2.warpAffine(img,M,(rows,cols))
    return img_T

def panning(img,x,y):#M = np.float32([[1, 0, 200], [0, 1, 100]])
    M = np.float32([[1, 0, x], [0, 1, y]])
    cols, rows=img.shape[:2]
    img_T = cv2.warpAffine(img, M, (rows,cols))
    return img_T

def flip(img,flipcode):#0垂直，1水平，-1垂直水平
    img_T=cv2.flip(img, flipcode)
    return img_T

def scale(img,scalecode):#scalecode -1缩小，1扩大
    cols, rows=img.shape[:2]
    if scalecode==1:
        img_T=cv2.resize(img,(rows,cols)*2)[:rows,:cols]
    else:
        img_T=np.zeros_like(img)
        img_T[cols//4:cols//4+cols//2,rows//4:rows//4+rows//2]=cv2.resize(img,(rows//2,cols//2))
    return img_T
        
def Contrast_and_Brightness(alpha, beta, img):
    blank = np.zeros(img.shape, img.dtype)
    # dst = alpha * img + beta * blank
    img_T = cv2.addWeighted(img, alpha, blank, 1-alpha, beta)
    return img_T
def crop(img,ratio):
    cols, rows=img.shape[:2]
    img_T=img[cols//2-int(cols*ratio[0])//2:cols//2+int(cols*ratio[0])//2,
              rows//2-int(rows*ratio[1])//2:rows//2+int(rows*ratio[1])//2]
    return img_T
def gamma(img, gamma):#gamma[0.5,1.5]
    invGamma = 1.0/gamma
    img=np.array(img).astype("uint8")
    table = []
    for i in range(256):
        table.append(((i / 255.0) ** invGamma) * 255)
    table = np.array(table).astype("uint8")
    return cv2.LUT(img, table)


from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
def elastic_transform(image,label, alpha, sigma, alpha_affine=None, random_state=None):
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]

    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    label = cv2.warpAffine(label, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape),map_coordinates(label, indices, order=1, mode='reflect').reshape(shape)
    
    
    
def object_transform(img,lab,case):
    if case==0:
        img_T,lab_T=elastic_transform(img,lab,img.shape[1]*3,
                                                          img.shape[1]*0.05,
                                                          img.shape[1]*0.09)
    elif case==1:
        x=np.random.randint(10,30)
        y=np.random.randint(10,30)
        img_T=panning(img,x,y)
        lab_T=panning(lab,x,y)
    elif case==2:
        angle=np.random.randint(-10,10)
        img_T=rotate(img,angle)
        lab_T=rotate(lab,angle)
    elif case==4:
        img_T=scale(img,-1)
        lab_T=scale(lab,-1)
    elif case==3:
        flipcode=np.random.randint(-1,1)
        img_T=flip(img,flipcode)
        lab_T=flip(lab,flipcode)
    
    return img_T,lab_T

def image_transform(img,lab,case):
    if case==0:
        flipcode=np.random.randint(-1,1)
        
        img_T=flip(img,flipcode)
        lab_T=flip(lab,flipcode)
    elif case==1:
        ratio=[1-np.random.rand()/5,1-np.random.rand()/5]
        img_T=crop(img,ratio)
        lab_T=crop(lab,ratio)
    elif case==2:
        gammacode=(np.random.rand()-0.5)/3+1
        img=img/img.max()*255
        img_T=gamma(img,gammacode)
        lab_T=lab
    elif case==3:
        img_T,lab_T=elastic_transform(img,lab,img.shape[1]*3,
                                                          img.shape[1]*0.05,
                                                          img.shape[1]*0.09)
    
    return img_T,lab_T
    

In [23]:
#Fusion
import cv2
def soft_fusion(source_img,source_lab,soft_img,soft_lab):

    soft_img=cv2.resize(soft_img,(source_img.shape[1],source_img.shape[0]))
    soft_lab=cv2.resize(soft_lab,(source_lab.shape[1],source_img.shape[0]))
    soft_lab=(soft_lab/soft_lab.max())
    mask=1-(soft_lab/soft_lab.max())
    
    aug_img=soft_img+source_img*mask
    aug_lab=source_lab*255+cv2.threshold(soft_lab,0.95,255,cv2.THRESH_BINARY)[1]
    
    #cv2.imwrite("D:\\dataaug\\US\\data_new\\benign\\aug_image\\source_image.png",source_img)
    #cv2.imwrite("D:\\dataaug\\US\\data_new\\benign\\aug_image\\soft_image.png",soft_img)
    #cv2.imwrite("D:\\dataaug\\US\\data_new\\benign\\aug_image\\soft_label.png",soft_lab)
    
    return aug_img,aug_lab
       

In [24]:
#Fusion
import cv2
def hard_fusion(source_img,source_lab,soft_img,soft_lab):
    
    soft_img=cv2.resize(soft_img,(source_img.shape[1],source_img.shape[0]))
    soft_lab=cv2.resize(soft_lab,(source_lab.shape[1],source_img.shape[0]))
    soft_lab=soft_lab/soft_lab.max()
    soft_lab=cv2.threshold(soft_lab,0.95,255,cv2.THRESH_BINARY)[1]
    
    mask=1-(soft_lab/soft_lab.max())
    
    aug_img=soft_img*(soft_lab/soft_lab.max())+source_img*mask
    aug_lab=source_lab+cv2.threshold(soft_lab,0.95,255,cv2.THRESH_BINARY)[1]
    
    #cv2.imwrite("D:\\dataaug\\US\\data_new\\benign\\aug_image\\source_image.png",source_img)
    #cv2.imwrite("D:\\dataaug\\US\\data_new\\benign\\aug_image\\soft_image.png",soft_img)
    #cv2.imwrite("D:\\dataaug\\US\\data_new\\benign\\aug_image\\soft_label.png",soft_lab)
    
    return aug_img,aug_lab
       

In [30]:
#读取图片路径
import numpy as np
def data_aug(source_img_path,source_lab_path,softimg_path,softlab_path,num):
    soft_paths=path_zip(softimg_path,softlab_path)
    source_paths=path_zip(source_img_path,source_lab_path)

    #挑选source和soft
    i=0
    for j in range(num):
        #soft_CP
        source_pair=Random_select(source_paths)
        soft_pair=Random_select(soft_paths)
        
        source_img=cv2.imread(source_pair[0])
        source_lab=cv2.imread(source_pair[1])
        source_lab=source_lab/source_lab.max()
        #source_lab=np.zeros_like(source_img)

        soft_img=cv2.imread(soft_pair[0])
        soft_lab=cv2.imread(soft_pair[1])
        soft_lab=soft_lab/soft_lab.max()
        #soft_img=scale(soft_img,-1)
        #soft_lab=scale(soft_img,-1)
        soft_hard_lab=cv2.threshold(soft_lab,0.95,1,cv2.THRESH_BINARY)[1]
        soft_hard_lab=cv2.resize(soft_hard_lab,(source_lab.shape[1],source_lab.shape[0]))
        
        if np.sum(source_lab*soft_hard_lab)<1:
            i=i+1
            source_lab=source_lab*255
            soft_lab=soft_lab*255
        #soft_lab transform
            soft_T=[np.random.randint(1,4),np.random.randint(1,4)]
            for case in soft_T:
                soft_img,soft_lab=object_transform(soft_img,soft_lab,case)

            hard_lab=cv2.threshold(soft_lab/soft_lab.max(),0.99,1,cv2.THRESH_BINARY)[1]
            kernel=np.ones((3,3),np.uint8)
            #hard_lab=cv2.dilate(hard_lab,kernel,iterations=3)
            hard_img=hard_lab*soft_img


            #soft_CP
            aug_softimg,aug_softlab=soft_fusion(source_img,source_lab,soft_img,soft_lab)
            aug_softimg=aug_softimg
            image_T=[np.random.randint(0,3),np.random.randint(0,3)]
            for case in image_T:
                aug_softimg,aug_softlab=image_transform(aug_softimg,aug_softlab,case)
            #aug_softimg=cv2.GaussianBlur(aug_softimg,(5,5),0)
            aug_softimg=cv2.resize(aug_softimg,(512,512))
            aug_softlab=cv2.resize(aug_softlab,(512,512))
            cv2.imwrite("D:\\dataaug\\paper\\figure3\\AUG\\aug_softimage\\"+str(i)+"_augSimage.jpg",aug_softimg)
            cv2.imwrite("D:\\dataaug\\paper\\figure3\\AUG\\aug_softlabel\\"+str(i)+"_augSlabel.jpg",aug_softlab)

            #TumorCP
            aug_hardimg,aug_hardlab=hard_fusion(source_img,source_lab,hard_img,hard_lab)
            aug_hardimg=0.7*aug_hardimg+0.3*source_img
            image_T=[np.random.randint(0,3),np.random.randint(0,3)]
            #for case in image_T:
            #    aug_hardimg,aug_hardlab=image_transform(aug_hardimg,aug_hardlab,case)
            aug_hardimg=cv2.GaussianBlur(aug_hardimg,(5,5),0)
            #cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\aug_hardimage\\"+str(i)+"_augHimage.jpg",aug_hardimg)
            #cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\aug_hardlabel\\"+str(i)+"_augHlabel.jpg",aug_hardlab)

            #Transform
            
            """
            S_image=source_img
            S_label=source_lab
            image_T=[np.random.randint(0,3),np.random.randint(0,3)]
            for case in image_T:
                S_image,S_label=image_transform(S_image,S_label,case)
            cv2.imwrite("D:\\dataaug\\US\\data_new\\malignant\\aug_transimage\\"+str(i)+"_augTimage.png",S_image)
            cv2.imwrite("D:\\dataaug\\US\\data_new\\malignant\\aug_translabel\\"+str(i)+"_augTlabel.png",S_label)
            
            """
           


        
        
        

In [31]:
data_aug(source_img_path,source_lab_path,softimg_path,softlab_path,50)

In [10]:
#ISIC粘贴背景
import numpy as np
source_img_path="D:\\dataaug\\ISIC\\ISIC\\background"
source_lab_path="D:\\dataaug\\ISIC\\ISIC\\background"
softlab_path="D:\\dataaug\\ISIC\\ISIC\\labels"
softimg_path="D:\\dataaug\\ISIC\\ISIC\\images"

def ISIC_aug(source_img_path,source_lab_path,softimg_path,softlab_path,num):
    soft_paths=path_zip(softimg_path,softlab_path)
    source_paths=path_zip(source_img_path,source_lab_path)

    #挑选source和soft
    i=0
    for j in range(num):
        i=i+1
        #soft_CP
        source_pair=Random_select(source_paths)
        soft_pair=Random_select(soft_paths)
        
        source_img=cv2.imread(source_pair[0])

        soft_img=cv2.imread(soft_pair[0])
        soft_lab=cv2.imread(soft_pair[1])
        
        alpha=0.5
        
        aug_softimg=alpha*source_img+(1-alpha)*soft_img
        aug_softlab=soft_lab

       

        cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\aug_image\\"+str(i)+"_augimage.png",aug_softimg)
        #cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\aug_image\\"+str(i)+"_augimage1.png",soft_img)
        cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\aug_label\\"+str(i)+"_auglabel.png",aug_softlab)


        
        

In [104]:
ISIC_aug(source_img_path,source_lab_path,softimg_path,softlab_path,2000)

In [1]:
import numpy as np

In [6]:
a=np.asarray(-1).astype(np.bool)
a

array(True)

In [57]:
flipcode=np.random.randint(-1,2)

In [58]:
flipcode

1

In [75]:
import cv2
img1=cv2.imread("D:\\dataaug\\ISIC\\ISIC\\23.jpg")
img2=cv2.imread("D:\\dataaug\\ISIC\\ISIC\\1_image.jpg")
mask=cv2.imread("D:\\dataaug\\ISIC\\ISIC\\1_mask.png")

In [76]:
mask=1-mask/mask.max()

In [77]:
img3=0.5*img1+0.5*img2

In [78]:
cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\test.jpg",img3)

True

In [101]:
img=cv2.imread("D:\\dataaug\\ISIC\\ISIC\\background\\14.jpg")
for i in range(30):
    cv2.imwrite("D:\\dataaug\\ISIC\\ISIC\\background\\"+str(100*i+1)+".jpg",img)